<a href="https://colab.research.google.com/github/MayerT1/COLAB_DEV_GEDI-BDL/blob/main/Neon_tree_height_and_Lidar_CHM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.neonscience.org/resources/learning-hub/tutorials/tree-heights-veg-structure-chm

R and Python Guide: https://www.neonscience.org/resources/learning-hub/tutorials/neondatastackr

Make sure this is set up Runtime-> Change Runtime Type -> Run Time Type -> R

# 1. Setup

In [1]:
install.packages("neonUtilities")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
install.packages("neonOS")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
install.packages("terra")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
install.packages("devtools")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [5]:
install.packages("dplyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
devtools::install_github("NEONScience/NEON-geolocation/geoNEON")

Skipping install of 'geoNEON' from a github remote, the SHA1 (59bdef7f) has not changed since last install.
  Use `force = TRUE` to force installation



Libraries

In [7]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [8]:
library(terra)

terra 1.8.21



In [9]:
library(neonUtilities)

In [10]:
library(neonOS)

In [11]:
library(geoNEON)

In [11]:
options(stringsAsFactors=F)

# set working directory

# adapt directory path for your system

# wd <- "~/data"

# setwd(wd)

In [12]:
wd <- "~/data"


In [13]:
getwd()

[1] "/content"

In [ ]:
#setwd(wd)

# 2. Vegetation structure data

[Cheatsheet](https://www.neonscience.org/sites/default/files/cheat-sheet-neonUtilities.pdf)

In [14]:
veglist <- loadByProduct(dpID="DP1.10098.001",
                         site="GUAN", #WREF
                         package="basic",
                         release="RELEASE-2023",
                         check.size = FALSE)

Finding available files
  |======================================================================| 100%

  |======================================================================| 100%

Unpacking zip files using 1 cores.
Stacking operation across a single core.
Stacking table vst_mappingandtagging
Stacking table vst_perplotperyear
Stacking table vst_shrubgroup
Stacking table vst_non-woody
Stacking table vst_apparentindividual
Copied the most recent publication of validation file to /stackedFiles
Copied the most recent publication of categoricalCodes file to /stackedFiles
Copied the most recent publication of variable definition file to /stackedFiles
Finished: Stacked 5 data tables and 4 metadata tables!
Stacking took 2.688707 secs


In [15]:
head(veglist)

name,pubCode,description,startDate,endDate
<chr>,<chr>,<chr>,<chr>,<chr>
Identification Qualifiers,cf. species,roughly equals but 'not sure' about the species,2012-01-01T00:00:00Z,
Identification Qualifiers,aff. species,similar to but is not the species,2012-01-01T00:00:00Z,
Identification Qualifiers,cf. genus,roughly equals but 'not sure' about the genus,2012-01-01T00:00:00Z,
Identification Qualifiers,aff. genus,similar to but is not the genus,2012-01-01T00:00:00Z,
Identification Qualifiers,cf. subspecies,roughly equals but 'not sure' about the subspecies,2012-01-01T00:00:00Z,
Identification Qualifiers,aff. subspecies,similar to but is not the subspecies,2012-01-01T00:00:00Z,
Identification Qualifiers,cf. family,roughly equals but 'not sure' about the family,2012-01-01T00:00:00Z,
Identification Qualifiers,aff. family,similar to but is not the family,2012-01-01T00:00:00Z,
Identification Qualifiers,cf. variety,roughly equals but 'not sure' about the variety,2012-01-01T00:00:00Z,


In [16]:
attributes(veglist)

$names
 [1] "categoricalCodes_10098"      "citation_10098_RELEASE-2023"
 [3] "issueLog_10098"              "readme_10098"               
 [5] "validation_10098"            "variables_10098"            
 [7] "vst_apparentindividual"      "vst_mappingandtagging"      
 [9] "vst_non-woody"               "vst_perplotperyear"         
[11] "vst_shrubgroup"

In [17]:
vegmap <- getLocTOS(veglist$vst_mappingandtagging,
                          "vst_mappingandtagging")

[1] "Please note locations will be calculated only for mapped woody individuals. To find subplot locations for unmapped individuals, use this function with the vst_apparentindividual, vst_non-woody, and/or vst_shrubgroup tables."
  |==================================================================    |  94%

The following namedLocation was not found: GUAN_013.basePlot.vst.40



  |===================================================================   |  96%

The following namedLocation was not found: GUAN_003.basePlot.vst.40



  |======================================================================| 100%


In [18]:
df <- vegmap
glimpse(df)
write.csv(df, file='vegmap.csv')

Rows: 7,357
Columns: 38
$ uid                        <chr> "02d00b0f-39d2-40c3-b02d-bf988d8100d3", "9a…
$ namedLocation              <chr> "GUAN_044.basePlot.vst", "GUAN_043.basePlot…
$ date                       <dttm> 2013-04-04, 2015-05-12, 2015-05-12, 2015-0…
$ eventID                    <chr> "vst_GUAN_2017", "vst_GUAN_2015", "vst_GUAN…
$ domainID                   <chr> "D04", "D04", "D04", "D04", "D04", "D04", "…
$ siteID                     <chr> "GUAN", "GUAN", "GUAN", "GUAN", "GUAN", "GU…
$ plotID                     <chr> "GUAN_044", "GUAN_043", "GUAN_043", "GUAN_0…
$ subplotID                  <chr> "39", "39", "39", "39", "39", "39", "39", "…
$ nestedSubplotID            <chr> NA, "1", "1", "1", "1", "1", "1", "1", "1",…
$ pointID                    <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ stemDistance               <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ stemAzimuth                <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ recordType    

In [ ]:
head(vegmap)

In [ ]:
attributes(vegmap)

In [ ]:
veg <- joinTableNEON(veglist$vst_apparentindividual,
                     vegmap,
                     name1="vst_apparentindividual",
                     name2="vst_mappingandtagging")

In [ ]:
head(veg)

In [ ]:
attributes(veg)

In [ ]:
symbols(veg$adjEasting[which(veg$plotID=="WREF_075")],
        veg$adjNorthing[which(veg$plotID=="WREF_075")],
        circles=veg$stemDiameter[which(veg$plotID=="WREF_075")]/100/2,
        inches=F, xlab="Easting", ylab="Northing")

In [ ]:
symbols(veg$adjEasting[which(veg$plotID=="WREF_075")],
        veg$adjNorthing[which(veg$plotID=="WREF_075")],
        circles=veg$stemDiameter[which(veg$plotID=="WREF_075")]/100/2,
        inches=F, xlab="Easting", ylab="Northing")

symbols(veg$adjEasting[which(veg$plotID=="WREF_075")],
        veg$adjNorthing[which(veg$plotID=="WREF_075")],
        circles=veg$adjCoordinateUncertainty[which(veg$plotID=="WREF_075")],
        inches=F, add=T, fg="lightblue")

# Canopy height model data

In [ ]:
byTileAOP(dpID="DP3.30015.001", site="WREF", year="2017",
          easting=veg$adjEasting[which(veg$plotID=="WREF_075")],
          northing=veg$adjNorthing[which(veg$plotID=="WREF_075")],
          check.size=FALSE, savepath=wd)



chm <- rast(paste0(wd, "/DP3.30015.001/neon-aop-products/2017/FullSite/D16/2017_WREF_1/L3/DiscreteLidar/CanopyHeightModelGtif/NEON_D16_WREF_DP3_580000_5075000_CHM.tif"))

In [ ]:
plot(chm, col=topo.colors(5))

# 4. Comparing the two datasets

In [ ]:
vegsub <- veg[which(veg$adjEasting >= ext(chm)[1] &
                      veg$adjEasting <= ext(chm)[2] &
                      veg$adjNorthing >= ext(chm)[3] &
                      veg$adjNorthing <= ext(chm)[4]),]

In [ ]:
valCHM <- extract(chm,
                  cbind(vegsub$adjEasting,
                  vegsub$adjNorthing))



plot(valCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegsub$height, pch=20, xlab="Height",
     ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(valCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegsub$height, use="complete")

## [1] 0.3824467

In [ ]:
valCHMbuff <- extract(chm,
                  buffer(vect(cbind(vegsub$adjEasting,
                  vegsub$adjNorthing)),
                  width=vegsub$adjCoordinateUncertainty),
                  fun=max)



plot(valCHMbuff$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegsub$height, pch=20, xlab="Height",
     ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(valCHMbuff$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegsub$height, use="complete")

## [1] 0.3698753

map-centric approach

In [ ]:
easting10 <- 10*floor(vegsub$adjEasting/10)

northing10 <- 10*floor(vegsub$adjNorthing/10)

vegsub <- cbind(vegsub, easting10, northing10)

In [ ]:
attributes(vegsub)

In [ ]:
vegbin <- stats::aggregate(vegsub,
                           by=list(vegsub$easting10,
                                   vegsub$northing10),
                           FUN=max)

In [ ]:
CHM10 <- terra::aggregate(chm, fact=10, fun=max)

plot(CHM10, col=topo.colors(5))

In [ ]:
vegbin$easting10 <- vegbin$easting10 + 5

vegbin$northing10 <- vegbin$northing10 + 5

binCHM <- extract(CHM10, cbind(vegbin$easting10,
                               vegbin$northing10))

plot(binCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegbin$height, pch=20,
     xlab="Height", ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(binCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegbin$height, use="complete")

## [1] 0.2244228

tree-centric approach

In [ ]:
vegsub <- vegsub[order(vegsub$height,
                       decreasing=T),]

In [ ]:
vegfil <- vegsub

for(i in 1:nrow(vegsub)) {
    if(is.na(vegfil$height[i]))
        next
    dist <- sqrt((vegsub$adjEasting[i]-vegsub$adjEasting)^2 +
                (vegsub$adjNorthing[i]-vegsub$adjNorthing)^2)
    vegfil$height[which(dist<0.3*vegsub$height[i] &
                        vegsub$height<vegsub$height[i])] <- NA
}



vegfil <- vegfil[which(!is.na(vegfil$height)),]

In [ ]:
filterCHM <- extract(chm,
                     cbind(vegfil$adjEasting,
                           vegfil$adjNorthing))

plot(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegfil$height, pch=20,
     xlab="Height", ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegfil$height)

## [1] 0.8070586

In [ ]:
vegfil <- vegfil[which(vegfil$plantStatus=="Live"),]

filterCHM <- extract(chm,
                     cbind(vegfil$adjEasting,
                           vegfil$adjNorthing))

plot(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM~
       vegfil$height, pch=20,
     xlab="Height", ylab="Canopy height model")

lines(c(0,50), c(0,50), col="grey")

In [ ]:
cor(filterCHM$NEON_D16_WREF_DP3_580000_5075000_CHM,
    vegfil$height)

## [1] 0.9057883